In [0]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions._
import group.research.aging.spark.extensions._

In [1]:
//val homology_to_gene = spark.readTSV("/data/ensembl/99/website/intermediates/homology_to_gene.tsv", false).toDF("homology_id","gene_member_id")
//val gene_members = spark.readTSV("/data/ensembl/99/compara/dump/homology.txt", false)

In [2]:
val homologies_ortho = spark.readTSV("/data/ensembl/99/compara/dump/homology.txt", false).toDF(
    "homology_id","method_link_species_set_id",	"description",	"is_tree_compliant","dn",	"ds",	"n",	"s",	"lnl",	
    "species_tree_node_id",	"gene_tree_node_id",	"gene_tree_root_id",	"goc_score",	"wga_coverage", "is_high_confidence"
).select("homology_id", "method_link_species_set_id",	"description", "goc_score",	 "is_high_confidence")
homologies_ortho.columns.toList

List(homology_id, method_link_species_set_id, description, goc_score, is_high_confidence)

In [3]:
homologies_ortho.writeTSV("/data/ensembl/99/website/intermediates/homologies_ortho.tsv")
//homologies_one_to_one.writeTSV("/data/ensembl/99/website/intermediates/homologies_one_to_one.tsv")

parts of /data/ensembl/99/website/intermediates/homologies_ortho.tsv merged!


/data/ensembl/99/website/intermediates/homologies_ortho.tsv

In [12]:
homologies_ortho.writeParquet("/data/ensembl/99/website/intermediates/homologies_ortho.parquet", true)

parts of /data/ensembl/99/website/intermediates/homologies_ortho.parquet merged!


/data/ensembl/99/website/intermediates/homologies_ortho.parquet

In [4]:
val ortho_table = spark.readTSV("/data/ensembl/99/website/intermediates/ortho_table.tsv")
val ortho_table2 = ortho_table.join(homologies_ortho.select(	"homology_id", "description", "is_high_confidence"), "homology_id")
ortho_table2.writeParquet("/data/ensembl/99/website/intermediates/ortho_table2.parquet", true)
ortho_table2.writeTSV("/data/ensembl/99/website/intermediates/ortho_table2.tsv")

org.apache.spark.sql.AnalysisException: USING column `homology_id` cannot be resolved on the left side of the join. The left-side columns: [_c0, _c1, _c2, _c3];

In [5]:
val homologies.one_to_one = homologies.where($"description" === "ortholog_one2one").select(
    "homology_id",
"method_link_species_set_id",	
"description",	
"homology_description",	
"is_tree_compliant",
"dn	double precision",	
"ds	double precision",	
"n	double precision",	
"s	double precision",	
"lnl	double precision",	
"goc_score",	
"wga_coverage",
"is_high_confidence")

In [6]:
val homology_to_gene = spark.readTSV("/data/ensembl/99/compara/dump/homology_member.txt", false).toDF("homology_id","gene_member_id",	
"seq_member_id",	
"cigar_line",	
"perc_cov",	
"perc_id",
"perc_pos").select("homology_id","gene_member_id").writeTSV("/data/ensembl/99/website/intermediates/homology_to_gene.tsv", false)

parts of /data/ensembl/99/website/intermediates/homology_to_gene.tsv merged!


In [7]:
val soma = spark.readTSV("/data/other/ST16_SOMAscan_171_subjects_T.tsv", true)
soma.show()

+--------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------

In [8]:
val human_mapping = spark.read.parquet("/data/indexes/uniprot/uniprot_human_mapping.parquet")
human_mapping.show()

+----------+-----------+------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+---------------+-------------+--------------------+-----+--------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|uniprot_ac| uniprot_id|entrez|              refSeq|                  gi|                 pdb|                  go|       uniref100|       uniref90|       uniref50|      uniparc|                 pir|taxon|           mim|unigene|              pubmed|                embl|            embl_cds|             ensembl|         ensembl_trs|         ensembl_pro|   additional_pubmed|
+----------+-----------+------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+---------------+-------------+--------------------+-----+--------------+-------+--------------------

In [9]:
val soma_legend = spark.readTSV("/data/other/SOMA_legend.tsv", true)

In [10]:
val soma_legend_ext = human_mapping.withColumnRenamed("uniprot_ac", "Uniprot").select("Uniprot", "ensembl").join(soma_legend, "Uniprot")
soma_legend_ext.show()

+-------+--------------------+----------+----------+--------+--------------------+--------------------+------------+----------------+--------+-----+-------+
|Uniprot|             ensembl|      R_ID|     SeqId|  SomaId|      TargetFullName|              Target|EntrezGeneID|EntrezGeneSymbol|Organism|Units|   Type|
+-------+--------------------+----------+----------+--------+--------------------+--------------------+------------+----------------+--------+-----+-------+
| P31946|     ENSG00000166913|     YWHAB|14156-33_3|SL004815|14-3-3 protein be...|14-3-3 protein be...|        7529|           YWHAB|   Human|  RFU|Protein|
| P62258|ENSG00000108953; ...|     YWHAE|14157-21_3|SL004984|14-3-3 protein ep...|             14-3-3E|        7531|           YWHAE|   Human|  RFU|Protein|
| P31947|     ENSG00000175793|       SFN| 4829-43_6|SL001737|14-3-3 protein sigma|           STRATIFIN|        2810|             SFN|   Human|  RFU|Protein|
| P27348|     ENSG00000134308|     YWHAQ| 7625-27_3|SL0080

In [11]:
soma_legend_ext.writeTSV("/data/other/SOMA_legend_ext.tsv")

parts of /data/other/SOMA_legend_ext.tsv merged!


/data/other/SOMA_legend_ext.tsv